In [1]:
import keras
import tensorflow
import numpy as np
import pandas as pd
import load_data
import SimpleITK
np.random.seed(42) #Make sure results can be reproduced
from keras.models import Sequential #Feed forward
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Convolution3D, MaxPooling3D
from keras.utils import np_utils


labels = pd.read_csv('stage1_labels.csv') #load the CSV containing the labels
patient_id = '0a0c32c9e08cc2ea76a71649de56be6d' #Our selected patient ID
patient1 = load_data.get_lungs_arr(patient_id, 'patient_image/') #Use preprocessing code to obtain 64x64x64 3D image for our patient

Using TensorFlow backend.


In [2]:
X_train = patient1.reshape(1,64,64,64,1)
# Can be interpreted this way (number of samples, number of channels (1 for b/w 3 for colour), depth, width height)

y_train = np_utils.to_categorical(labels[labels['id']==patient_id]['cancer'].values, 2)
# Change labels to array. E.g. instead of having 0 or 1 we have [1,0] or [0,1] where the index of '1' is 
# the true label


In [6]:
model = Sequential() 
# Feed forward 1 by 1 hence sequential. Other architectures such as LSTM would be different

In [7]:
model.add(Convolution3D(8,10,10,10, activation='relu',input_shape=(64,64,64,1),dim_ordering='tf'))
model.output_shape
# Add layers to model. In this case our first layer is a 3D convolution layer. 
# The first 4 numbers represents (number of filters, filter d,w,h). 
# The activation is the function used to make the network non-linear. 
# Since this is the first layer of the model, we will need an input_shape parameter too. 
# dim_ordering is just the parameter telling the model that we use TensorFlow convention for the dimesion ordering 
# i.e. d,w,h,channels

(None, 55, 55, 55, 8)

In [8]:
model.add(Convolution3D(8,10,10,10,activation='relu',dim_ordering='tf'))
print model.output_shape
model.add(MaxPooling3D(pool_size=(2,2,2))) # Reduce number of parameters
print model.output_shape
model.add(Convolution3D(8,5,5,5,activation='relu',dim_ordering='tf'))
print model.output_shape
model.add(MaxPooling3D(pool_size=(2,2,2))) # Reduce number of parameters
print model.output_shape
model.add(Dropout(0.25)) # Regularised the model to prevent overfit
print model.output_shape
# Now we have 2x conv + 2 maxpool + 1 dropout
model.add(Flatten()) #Must flattern to pass into dense
print model.output_shape
model.add(Dense(128, activation='relu')) # 
print model.output_shape
model.add(Dropout(0.5))
print model.output_shape
model.add(Dense(2))
model.add(Activation('softmax')) # 2 ouput matches binary classification criteria
print model.output_shape

(None, 46, 46, 46, 8)
(None, 23, 23, 23, 8)
(None, 19, 19, 19, 8)
(None, 9, 9, 9, 8)
(None, 9, 9, 9, 8)
(None, 5832)
(None, 128)
(None, 128)
(None, 2)


In [9]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train)

Epoch 1/10
